just add alpha beta .... if bad results

In [1]:
import mne
import numpy as np
import os
import re

In [2]:
raw_files = set(filter(re.compile(".*.mff").match, os.listdir("./VRCP")))
fil_files = set(filter(re.compile(".*fil.mff").match, os.listdir("./VRCP")))
seg_files = set(filter(re.compile(".*seg.mff").match, os.listdir("./VRCP")))
mat_files = set(filter(re.compile(".*.mat").match, os.listdir("./VRCP")))
raw_files = raw_files - fil_files - seg_files - mat_files

In [3]:
print(fil_files)

{'VRCP-CYM-HarshaW-27-3-19_20190327_120035_fil.mff', 'VRCP-CYM-Atharva-April12_20190412_044115_fil.mff', 'VRCP-RGB-Atharva-April12_20190412_042629_fil.mff', 'VRCP-RGB-HarshaW-27-3-19_20190327_114603_fil.mff'}


In [ ]:
raw_iter = iter(sorted(raw_files))
fil_iter = iter(sorted(fil_files))
seg_iter = iter(sorted(seg_files))
mat_iter = iter(sorted(mat_files))

In [ ]:
rf = next(raw_iter)
ff = next(fil_iter)

In [ ]:
# raw_egi = mne.io.read_raw_egi("VRCP/" + rf)
# raw_egi.info

In [ ]:
%matplotlib inline
# raw_egi.plot(n_channels = len(raw_egi.ch_names))

In [ ]:
fil_egi = mne.io.read_raw_egi("VRCP/" + ff)
fil_egi.plot(n_channels = len(fil_egi.ch_names))

In [ ]:
fil_egi.info

In [ ]:
ev = mne.find_events(fil_egi)
ev = ev[:-1]

print(ev.shape)

In [ ]:
print(ev[:10])

In [ ]:
# ev2 = mne.event.shift_time_events(ev, [1, 2, 3], 1, fil_egi.info['sfreq'])
# print(ev2[:10])

In [ ]:
time_of_event = 5 # colour was shown for 5 seconds
num_splits = 5 # split each event into 5 splits
time_of_split = time_of_event/num_splits
shifted_events = []

for i in range(num_splits):
    ev_shift = mne.event.shift_time_events(ev, [1, 2, 3], i*time_of_split, fil_egi.info['sfreq'])
    shifted_events.append(ev_shift)

In [ ]:
shifted_events

In [ ]:
tmin = 0
tmax = 0.999 # for one split of 1 second

In [ ]:
ep2 = mne.Epochs(fil_egi, ev, tmin = tmin, tmax = 4.999, baseline = None, verbose = True)

In [ ]:
ep2.get_data().shape

In [ ]:
shifted_epochs = []

for i in range(num_splits):
    ep = mne.Epochs(fil_egi, shifted_events[i], tmin = tmin, tmax = tmax, baseline = None, verbose = True)
    shifted_epochs.append(ep)

In [ ]:
for i in range(num_splits):
    shifted_epochs[i].plot(scalings = 'auto', block = True)

In [ ]:
#relevant channels for the occipital lobe

channels = ['E9', 'E10', 'E20']

In [ ]:
# ep.plot(picks = channels, scalings = 'auto', n_epochs = len(ev), block = True)

In [ ]:
ep_picked = ep.load_data().pick(channels)
ep2_picked = ep2.load_data().pick_channels(channels)

In [ ]:
shifted_ep_pick = []
for i in range(num_splits):
    ep_pick = shifted_epochs[i].load_data().pick_channels(channels).filter(l_freq = 0, h_freq = 30, verbose = False)
    shifted_ep_pick.append(ep_pick)

In [ ]:
for i in range(num_splits):
    print(shifted_ep_pick[i].get_data().shape)

In [ ]:
ep_picked.plot(scalings = 'auto', n_epochs = 5, block = True)
ep2_picked.plot(scalings = 'auto', n_epochs = 5, block = True)

In [ ]:
ep_pick_fil = ep_picked.filter(l_freq = 0, h_freq = 30)

In [ ]:
ep_pick_fil.plot(scalings = 'auto', n_epochs = len(ev), block = True)

In [ ]:
ep.plot_psd_topomap(normalize = True)

In [ ]:
ep_picked.plot_psd_topomap(normalize = True)

In [ ]:
print(ep_picked.get_data().shape)
print(ep_pick_fil.get_data().shape)

In [ ]:
ep_pick_fil.get_data().shape

In [ ]:
psd, freq = mne.time_frequency.psd_multitaper(ep_picked, fmax = 30)

print(freq[:100])
print(psd.shape)
print(freq[-100:])
print(psd[0, 0, 3])

In [ ]:
shifted_ep_pick[0].get_data().shape

In [ ]:
psds = []
freqs = []
h_freq = 30
for i in range(num_splits):
    psd, freq = mne.time_frequency.psd_multitaper(shifted_ep_pick[i])
    
    h_freq_pos = np.searchsorted(freq, h_freq, side = 'left')
    psd = psd[:, :, :h_freq_pos]
    psds.append(psd)
    freqs.append(freq[:h_freq_pos])

In [ ]:
print(h_freq_pos)
print(psds[0].shape)


In [ ]:
print(freq.shape)
freq[:100]

In [ ]:
psd_1, freq_1 = mne.time_frequency.psd_multitaper(ep_pick_fil, fmax = 30.0)

# print(freq_1[:100])
print(psd_1.shape)
# print(freq_1[-100:])
# print(psd_1[0, 0, 250])
# print(freq_1[250])
print(np.searchsorted(freq, 30))
print(freq)
print(psd_1[0, 0, :])
# print(freq[150])

In [4]:
data_ = []
psd_data_ = []
freqs_ = []
labels_ = []

time_of_event = 5 # colour was shown for 5 seconds
num_splits = 5 # split each event into 5 splits
time_of_split = time_of_event/num_splits
h_freq = 30

channels = ['E9', 'E10', 'E20']

In [5]:
print(time_of_split)

1.0


In [6]:
# for ff in fil_files:
#     fil_egi = mne.io.read_raw_egi("VRCP/" + ff)
#     ev = mne.find_events(fil_egi)
#     ep = mne.Epochs(fil_egi, ev, tmin = tmin, tmax = tmax, baseline = None, verbose = True)
#     ep_picked = ep.load_data().pick(channels)
# #     ep_pick_fil = ep_picked.filter(l_freq = 0, h_freq = 70)
#     psd, freq = mne.time_frequency.psd_multitaper(ep_picked)
    
#     psd_data_.append(psd)
#     freqs_.append(np.expand_dims(freq, axis = 1))
#     data_.append(ep_picked.get_data())
#     labels_.append(ev[ep.selection, 2])

In [213]:
x = 0
num_ev_fil = np.zeros(4, dtype = np.int32)
num_ev_fil

array([0, 0, 0, 0], dtype=int32)

In [214]:
for ff in fil_files:
    fil_egi = mne.io.read_raw_egi("VRCP/" + ff, verbose = False)
    sfreq = fil_egi.info['sfreq']
    ev = mne.find_events(fil_egi, verbose = False)
    ev = ev[:-1] # dont use the last event as it may not be sampled for 5 seconds, epoch creation will give error
    
    tmin = 0
    tmax = time_of_split - 1/sfreq
    s = 0
    for i in range(num_splits):
        shifted_ev = mne.event.shift_time_events(ev, [1, 2, 3], i*time_of_split, sfreq)
        ep = mne.Epochs(fil_egi, shifted_ev, tmin = tmin, tmax = tmax, baseline = None, verbose = False)
        ep_pick = ep.load_data().pick(channels).filter(l_freq = 0, h_freq = 30)
#         ep_pick.plot()
        psd, freq = mne.time_frequency.psd_multitaper(ep_pick, fmax = 30, verbose = False)
        
        psd_data_.append(psd)
        freqs_.append(np.expand_dims(freq, axis = 1))
        labels_.append(shifted_ev[ep.selection, 2])
        s = s + len(ep.selection)
    num_ev_fil[x] = s
    x = x + 1;

In [215]:
print(num_ev_fil.sum())
print(num_ev_fil)
print(len(psd_data_), len(labels_))

1265
[295 285 390 295]
50 50


In [179]:
labels = np.concatenate(labels_[:5])
psd_data = np.concatenate(psd_data_[:5])

freqs = freq

In [180]:
print(labels.shape, psd_data.shape)

(295,) (295, 3, 31)


In [181]:
print(freqs)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]


In [182]:
# add features - sums of delta(0-4), theta(4-7), alpha(8-15), beta(16-31)

d_max = 4
t_max = 8
a_max = 14
b_max = 30

d_r = np.searchsorted(freqs, d_max, side = 'left') 
t_r = np.searchsorted(freqs, t_max, side = 'left') 
a_r = np.searchsorted(freqs, a_max, side = 'left') 
b_r = np.searchsorted(freqs, b_max, side = 'left') 

In [183]:
print(d_r, t_r, a_r, b_r)

4 8 14 30


In [184]:
print(psd_data.shape)

d_sums = np.expand_dims(psd_data[:, :, :d_r].sum(2), axis = 2)
t_sums = np.expand_dims(psd_data[:, :, d_r:t_r].sum(2), axis = 2)
a_sums = np.expand_dims(psd_data[:, :, t_r:a_r].sum(2), axis = 2)
b_sums = np.expand_dims(psd_data[:, :, a_r:b_r].sum(2), axis = 2)
print(d_sums.shape, t_sums.shape, a_sums.shape, b_sums.shape)
band_sums = np.concatenate([d_sums, t_sums, a_sums, b_sums], axis = 2)
print(band_sums.shape)

psd_band_data = np.concatenate([psd_data, band_sums], axis = 2)
print(psd_band_data.shape)

(295, 3, 31)
(295, 3, 1) (295, 3, 1) (295, 3, 1) (295, 3, 1)
(295, 3, 4)
(295, 3, 35)


In [185]:
labels_r = labels[labels == 1]
labels_g = labels[labels == 2]
labels_b = labels[labels == 3]

In [186]:
psd_data_r = psd_data[labels == 1, :, :]
psd_data_g = psd_data[labels == 2, :, :]
psd_data_b = psd_data[labels == 3, :, :]

psd_band_data_r = psd_band_data[labels == 1, :, :]
psd_band_data_g = psd_band_data[labels == 2, :, :]
psd_band_data_b = psd_band_data[labels == 3, :, :]

In [187]:
print(labels_r.shape, labels_g.shape, labels_b.shape, psd_data_r.shape, psd_data_g.shape, psd_data_b.shape)

(100,) (100,) (95,) (100, 3, 31) (100, 3, 31) (95, 3, 31)


In [188]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.utils import to_categorical

In [189]:
labels_cat = to_categorical(labels)
labels_cat = labels_cat[:, 1:]
labels_cat.shape

(295, 3)

In [190]:
from sklearn.model_selection import train_test_split

# should give error because psd_data
# psd_train, psd_test, labels_train, labels_test = train_test_split(psd_data, labels_cat, random_state = 42, test_size = 0.2)

psd_train_r, psd_test_r, labels_train_r, labels_test_r = train_test_split(psd_data_r, labels_cat[labels == 1, :], random_state = 42, test_size = 0.2)
psd_train_g, psd_test_g, labels_train_g, labels_test_g = train_test_split(psd_data_g, labels_cat[labels == 2, :], random_state = 42, test_size = 0.2)
psd_train_b, psd_test_b, labels_train_b, labels_test_b = train_test_split(psd_data_b, labels_cat[labels == 3, :], random_state = 42, test_size = 0.2)

psd_band_train_r, psd_band_test_r, labels_band_train_r, labels_band_test_r = train_test_split(psd_band_data_r, labels_cat[labels == 1, :], random_state = 42, test_size = 0.2)
psd_band_train_g, psd_band_test_g, labels_band_train_g, labels_band_test_g = train_test_split(psd_band_data_g, labels_cat[labels == 2, :], random_state = 42, test_size = 0.2)
psd_band_train_b, psd_band_test_b, labels_band_train_b, labels_band_test_b = train_test_split(psd_band_data_b, labels_cat[labels == 3, :], random_state = 42, test_size = 0.2)

In [191]:
print(labels_cat.shape)

(295, 3)


In [192]:
psd_train = np.concatenate([psd_train_r, psd_train_g, psd_train_b])
psd_test = np.concatenate([psd_test_r, psd_test_g, psd_test_b])
labels_train = np.concatenate([labels_train_r, labels_train_g, labels_train_b])
labels_test = np.concatenate([labels_test_r, labels_test_g, labels_test_b])

psd_band_train = np.concatenate([psd_band_train_r, psd_band_train_g, psd_band_train_b])
psd_band_test = np.concatenate([psd_band_test_r, psd_band_test_g, psd_band_test_b])
labels_band_train = np.concatenate([labels_band_train_r, labels_band_train_g, labels_band_train_b])
labels_band_test = np.concatenate([labels_band_test_r, labels_band_test_g, labels_band_test_b])

In [193]:
from sklearn.utils import shuffle

psd_train, labels_train = shuffle(psd_train, labels_train)
psd_test, labels_test = shuffle(psd_test, labels_test)

psd_band_train, labels_band_train = shuffle(psd_band_train, labels_band_train)
psd_band_test, labels_band_test = shuffle(psd_band_test, labels_band_test)

In [194]:
print(psd_train.shape, psd_test.shape, labels_train.shape, labels_test.shape)

(236, 3, 31) (59, 3, 31) (236, 3) (59, 3)


In [195]:
print(psd_band_train.shape, psd_band_test.shape, labels_band_train.shape, labels_band_test.shape)

(236, 3, 35) (59, 3, 35) (236, 3) (59, 3)


In [196]:
from sklearn.preprocessing import RobustScaler

n_ep_train, n_ch, n_f = psd_train.shape
n_ep_test = psd_test.shape[0]

scale = RobustScaler()
psd_train_norm = scale.fit_transform(psd_train.reshape((n_ep_train, n_ch*n_f))).reshape((n_ep_train, n_ch, n_f))
psd_test_norm = scale.transform(psd_test.reshape((n_ep_test, n_ch*n_f))).reshape((n_ep_test, n_ch, n_f))

n_ep_band_train, n_band_ch, n_band_f = psd_band_train.shape
n_ep_band_test = psd_band_test.shape[0]

scale_band = RobustScaler()
psd_band_train_norm = scale_band.fit_transform(psd_band_train.reshape((n_ep_band_train, n_band_ch*n_band_f))).reshape((n_ep_band_train, n_band_ch, n_band_f))
psd_band_test_norm = scale_band.transform(psd_band_test.reshape((n_ep_band_test, n_band_ch*n_band_f))).reshape((n_ep_band_test, n_band_ch, n_band_f))

In [197]:
print(psd_train_norm.shape, psd_test_norm.shape)
print(psd_band_train_norm.shape, psd_band_test_norm.shape,)

(236, 3, 31) (59, 3, 31)
(236, 3, 35) (59, 3, 35)


In [198]:
model = Sequential()
model.add(Flatten())
model.add(Dense(64, activation = 'relu')) # , kernel_regularizer = keras.regularizers.l1(0.01)))
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(16, activation = 'relu'))
model.add(BatchNormalization())
# model.add(LeakyReLU())
model.add(Dense(3, activation = 'softmax'))

In [199]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [200]:
model.fit(psd_train_norm, labels_train, batch_size = 16, epochs = 100, verbose = 2, validation_split = 0.2)

Train on 188 samples, validate on 48 samples
Epoch 1/100
 - 1s - loss: 1.3517 - acc: 0.4149 - val_loss: 1.1112 - val_acc: 0.4167
Epoch 2/100
 - 0s - loss: 1.3811 - acc: 0.3989 - val_loss: 1.0918 - val_acc: 0.4375
Epoch 3/100
 - 0s - loss: 1.2034 - acc: 0.4255 - val_loss: 1.2034 - val_acc: 0.4167
Epoch 4/100
 - 0s - loss: 1.2833 - acc: 0.4149 - val_loss: 1.1469 - val_acc: 0.3750
Epoch 5/100
 - 0s - loss: 1.2614 - acc: 0.4096 - val_loss: 1.0935 - val_acc: 0.3333
Epoch 6/100
 - 0s - loss: 1.2282 - acc: 0.3723 - val_loss: 1.0818 - val_acc: 0.3958
Epoch 7/100
 - 0s - loss: 1.1505 - acc: 0.4149 - val_loss: 1.1179 - val_acc: 0.3958
Epoch 8/100
 - 0s - loss: 1.1259 - acc: 0.3883 - val_loss: 1.1806 - val_acc: 0.3958
Epoch 9/100
 - 0s - loss: 1.3055 - acc: 0.3564 - val_loss: 1.2504 - val_acc: 0.3750
Epoch 10/100
 - 0s - loss: 1.0926 - acc: 0.4681 - val_loss: 1.2073 - val_acc: 0.3958
Epoch 11/100
 - 0s - loss: 1.1922 - acc: 0.4043 - val_loss: 1.1646 - val_acc: 0.3542
Epoch 12/100
 - 0s - loss: 1.

Epoch 97/100
 - 0s - loss: 0.8788 - acc: 0.6223 - val_loss: 1.0297 - val_acc: 0.4583
Epoch 98/100
 - 0s - loss: 0.8283 - acc: 0.6489 - val_loss: 1.0435 - val_acc: 0.4375
Epoch 99/100
 - 0s - loss: 0.7864 - acc: 0.6596 - val_loss: 1.0355 - val_acc: 0.4792
Epoch 100/100
 - 0s - loss: 0.8223 - acc: 0.6277 - val_loss: 1.1134 - val_acc: 0.4375


In [201]:
print(model.evaluate(psd_test_norm, labels_test))

59/59 [==============================] - 0s 148us/step
[1.024133544857219, 0.406779655460584]


In [202]:
print(model.evaluate(psd_train_norm[:50, :, :], labels_train[:50, :]))

50/50 [==============================] - 0s 157us/step
[0.79592777967453, 0.7200000047683716]


In [203]:
pred_class_test = model.predict_classes(psd_test_norm)

In [204]:
# pred_class_test = model.predict_classes([np.random.random((1, 3, 31))])

In [205]:
print(pred_class_test + 1)

[1 1 1 3 2 1 1 2 2 3 1 2 1 1 3 3 2 1 1 2 3 1 2 1 2 2 3 2 2 2 2 1 2 2 3 2 2
 3 1 1 3 1 2 2 3 1 1 1 3 3 2 2 1 2 1 2 3 2 2]


In [206]:
print(labels_test.argmax(1) + 1)

[3 3 3 3 1 1 2 2 1 3 2 2 3 1 3 3 2 1 2 1 1 3 3 2 2 1 3 1 2 1 2 2 1 2 2 1 2
 2 2 1 3 3 3 1 3 1 1 3 1 3 1 2 2 1 2 3 1 2 3]


In [207]:
model_2 = Sequential()
model_2.add(Flatten())
model_2.add(Dense(64, activation = 'relu')) # , kernel_regularizer = keras.regularizers.l1(0.01)))
# model.add(BatchNormalization())
model_2.add(Dropout(0.5))
model_2.add(Dense(16, activation = 'relu'))
model_2.add(BatchNormalization())
# model.add(LeakyReLU())
model_2.add(Dense(3, activation = 'softmax'))

In [208]:
model_2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [209]:
model_2.fit(psd_band_train_norm, labels_band_train, batch_size = 16, epochs = 100, verbose = 2, validation_split = 0.2)

Train on 188 samples, validate on 48 samples
Epoch 1/100
 - 2s - loss: 1.5266 - acc: 0.3404 - val_loss: 1.2564 - val_acc: 0.3125
Epoch 2/100
 - 0s - loss: 1.5040 - acc: 0.3404 - val_loss: 1.2496 - val_acc: 0.3125
Epoch 3/100
 - 0s - loss: 1.3164 - acc: 0.4043 - val_loss: 1.1935 - val_acc: 0.3333
Epoch 4/100
 - 0s - loss: 1.3224 - acc: 0.3830 - val_loss: 1.1697 - val_acc: 0.3542
Epoch 5/100
 - 0s - loss: 1.2497 - acc: 0.4043 - val_loss: 1.1695 - val_acc: 0.3333
Epoch 6/100
 - 0s - loss: 1.2075 - acc: 0.3830 - val_loss: 1.1441 - val_acc: 0.3125
Epoch 7/100
 - 0s - loss: 1.1396 - acc: 0.4415 - val_loss: 1.1156 - val_acc: 0.2917
Epoch 8/100
 - 0s - loss: 1.1725 - acc: 0.4096 - val_loss: 1.1349 - val_acc: 0.3333
Epoch 9/100
 - 0s - loss: 1.1268 - acc: 0.4309 - val_loss: 1.1582 - val_acc: 0.3542
Epoch 10/100
 - 0s - loss: 1.1777 - acc: 0.4096 - val_loss: 1.1300 - val_acc: 0.3333
Epoch 11/100
 - 0s - loss: 1.0384 - acc: 0.5053 - val_loss: 1.1387 - val_acc: 0.3542
Epoch 12/100
 - 0s - loss: 1.

Epoch 97/100
 - 0s - loss: 0.8599 - acc: 0.5957 - val_loss: 1.0667 - val_acc: 0.3542
Epoch 98/100
 - 0s - loss: 0.8089 - acc: 0.6330 - val_loss: 1.0787 - val_acc: 0.3542
Epoch 99/100
 - 0s - loss: 0.8141 - acc: 0.6277 - val_loss: 1.0897 - val_acc: 0.3542
Epoch 100/100
 - 0s - loss: 0.8174 - acc: 0.6117 - val_loss: 1.1027 - val_acc: 0.3125


In [210]:
model_2.evaluate(psd_band_test_norm, labels_band_test)

59/59 [==============================] - 0s 135us/step


[0.9550503997479455, 0.5084745777865588]

In [165]:
from sklearn.svm import SVC

In [166]:
svm_model = SVC(kernel = 'linear')

svm_model.fit(psd_train_norm.reshape((n_ep_train, n_ch*n_f)), labels_train.argmax(1))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [167]:
svm_model.predict(psd_test_norm.reshape(n_ep_test, n_ch*n_f))

array([0, 1, 2, 1, 2, 2, 2, 0, 2, 1, 0, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 1, 0, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 0, 1, 2, 1, 0, 2, 2, 2, 1])

In [168]:
svm_model.score(psd_test_norm.reshape(n_ep_test, n_ch*n_f), labels_test.argmax(1))

0.3898305084745763

In [169]:
svm_model_2 = SVC(kernel = 'linear', gamma = 'scale')

svm_model_2.fit(psd_band_train_norm.reshape(n_ep_band_train, n_band_ch*n_band_f), labels_band_train.argmax(1))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [171]:
svm_model_2.score(psd_band_test_norm.reshape(n_ep_band_test, n_band_ch*n_band_f), labels_band_test.argmax(1))

0.3898305084745763

In [172]:
svm_model_2.predict(psd_band_test_norm.reshape(n_ep_band_test, n_band_ch*n_band_f))

array([2, 2, 2, 0, 0, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2,
       2, 1, 2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 0, 2, 1, 1, 1, 2, 1, 2, 2, 2,
       2, 0, 2, 0, 2, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2])

In [173]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(loss_function = 'MultiClass')
cat_model.fit(psd_train_norm.reshape((n_ep_train, n_ch*n_f)), labels_train.argmax(1))

0:	learn: 1.0953605	total: 331ms	remaining: 5m 30s
1:	learn: 1.0934004	total: 706ms	remaining: 5m 52s
2:	learn: 1.0904853	total: 1.04s	remaining: 5m 45s
3:	learn: 1.0882272	total: 1.38s	remaining: 5m 42s
4:	learn: 1.0855868	total: 2.2s	remaining: 7m 17s
5:	learn: 1.0813034	total: 3.23s	remaining: 8m 54s
6:	learn: 1.0778831	total: 3.71s	remaining: 8m 45s
7:	learn: 1.0742647	total: 4.09s	remaining: 8m 27s
8:	learn: 1.0708795	total: 4.42s	remaining: 8m 7s
9:	learn: 1.0679321	total: 4.83s	remaining: 7m 58s
10:	learn: 1.0642502	total: 5.17s	remaining: 7m 44s
11:	learn: 1.0605446	total: 5.47s	remaining: 7m 30s
12:	learn: 1.0569021	total: 5.77s	remaining: 7m 18s
13:	learn: 1.0542204	total: 6.1s	remaining: 7m 9s
14:	learn: 1.0504250	total: 6.41s	remaining: 7m
15:	learn: 1.0474279	total: 6.71s	remaining: 6m 52s
16:	learn: 1.0448586	total: 7.04s	remaining: 6m 47s
17:	learn: 1.0433427	total: 7.35s	remaining: 6m 41s
18:	learn: 1.0413037	total: 7.65s	remaining: 6m 34s
19:	learn: 1.0377722	total: 7.

158:	learn: 0.7553264	total: 53.2s	remaining: 4m 41s
159:	learn: 0.7538636	total: 53.5s	remaining: 4m 41s
160:	learn: 0.7526823	total: 54s	remaining: 4m 41s
161:	learn: 0.7514615	total: 54.3s	remaining: 4m 40s
162:	learn: 0.7498641	total: 54.6s	remaining: 4m 40s
163:	learn: 0.7483009	total: 54.9s	remaining: 4m 40s
164:	learn: 0.7468765	total: 55.2s	remaining: 4m 39s
165:	learn: 0.7453681	total: 55.5s	remaining: 4m 38s
166:	learn: 0.7432677	total: 55.9s	remaining: 4m 38s
167:	learn: 0.7415061	total: 56.2s	remaining: 4m 38s
168:	learn: 0.7405599	total: 56.5s	remaining: 4m 37s
169:	learn: 0.7386797	total: 56.8s	remaining: 4m 37s
170:	learn: 0.7370204	total: 57.1s	remaining: 4m 36s
171:	learn: 0.7349315	total: 57.4s	remaining: 4m 36s
172:	learn: 0.7333875	total: 57.7s	remaining: 4m 35s
173:	learn: 0.7306160	total: 58s	remaining: 4m 35s
174:	learn: 0.7292344	total: 58.3s	remaining: 4m 34s
175:	learn: 0.7281317	total: 58.6s	remaining: 4m 34s
176:	learn: 0.7270047	total: 59s	remaining: 4m 34s

312:	learn: 0.5273984	total: 1m 43s	remaining: 3m 48s
313:	learn: 0.5265874	total: 1m 44s	remaining: 3m 47s
314:	learn: 0.5255616	total: 1m 44s	remaining: 3m 47s
315:	learn: 0.5245922	total: 1m 44s	remaining: 3m 47s
316:	learn: 0.5231009	total: 1m 45s	remaining: 3m 46s
317:	learn: 0.5219445	total: 1m 45s	remaining: 3m 46s
318:	learn: 0.5202829	total: 1m 45s	remaining: 3m 45s
319:	learn: 0.5187613	total: 1m 46s	remaining: 3m 45s
320:	learn: 0.5180719	total: 1m 46s	remaining: 3m 45s
321:	learn: 0.5171312	total: 1m 46s	remaining: 3m 44s
322:	learn: 0.5152324	total: 1m 47s	remaining: 3m 44s
323:	learn: 0.5135000	total: 1m 47s	remaining: 3m 44s
324:	learn: 0.5117004	total: 1m 47s	remaining: 3m 43s
325:	learn: 0.5103614	total: 1m 48s	remaining: 3m 43s
326:	learn: 0.5092468	total: 1m 48s	remaining: 3m 42s
327:	learn: 0.5074779	total: 1m 48s	remaining: 3m 42s
328:	learn: 0.5065532	total: 1m 48s	remaining: 3m 42s
329:	learn: 0.5055617	total: 1m 49s	remaining: 3m 41s
330:	learn: 0.5042075	total:

466:	learn: 0.3750478	total: 2m 32s	remaining: 2m 53s
467:	learn: 0.3740812	total: 2m 32s	remaining: 2m 53s
468:	learn: 0.3727703	total: 2m 33s	remaining: 2m 53s
469:	learn: 0.3720212	total: 2m 33s	remaining: 2m 52s
470:	learn: 0.3713912	total: 2m 33s	remaining: 2m 52s
471:	learn: 0.3706626	total: 2m 33s	remaining: 2m 52s
472:	learn: 0.3701863	total: 2m 34s	remaining: 2m 51s
473:	learn: 0.3691682	total: 2m 34s	remaining: 2m 51s
474:	learn: 0.3685861	total: 2m 34s	remaining: 2m 51s
475:	learn: 0.3676280	total: 2m 35s	remaining: 2m 50s
476:	learn: 0.3668924	total: 2m 35s	remaining: 2m 50s
477:	learn: 0.3663892	total: 2m 35s	remaining: 2m 50s
478:	learn: 0.3655008	total: 2m 36s	remaining: 2m 49s
479:	learn: 0.3644753	total: 2m 36s	remaining: 2m 49s
480:	learn: 0.3638955	total: 2m 36s	remaining: 2m 49s
481:	learn: 0.3631688	total: 2m 37s	remaining: 2m 48s
482:	learn: 0.3626562	total: 2m 37s	remaining: 2m 48s
483:	learn: 0.3620214	total: 2m 37s	remaining: 2m 48s
484:	learn: 0.3615925	total:

619:	learn: 0.2788950	total: 3m 23s	remaining: 2m 4s
620:	learn: 0.2782380	total: 3m 23s	remaining: 2m 4s
621:	learn: 0.2776779	total: 3m 24s	remaining: 2m 4s
622:	learn: 0.2772933	total: 3m 24s	remaining: 2m 3s
623:	learn: 0.2768139	total: 3m 24s	remaining: 2m 3s
624:	learn: 0.2764385	total: 3m 25s	remaining: 2m 3s
625:	learn: 0.2757262	total: 3m 25s	remaining: 2m 2s
626:	learn: 0.2753591	total: 3m 25s	remaining: 2m 2s
627:	learn: 0.2750201	total: 3m 26s	remaining: 2m 2s
628:	learn: 0.2747420	total: 3m 26s	remaining: 2m 1s
629:	learn: 0.2743313	total: 3m 26s	remaining: 2m 1s
630:	learn: 0.2737986	total: 3m 27s	remaining: 2m 1s
631:	learn: 0.2733339	total: 3m 27s	remaining: 2m
632:	learn: 0.2728303	total: 3m 27s	remaining: 2m
633:	learn: 0.2726387	total: 3m 27s	remaining: 2m
634:	learn: 0.2717122	total: 3m 28s	remaining: 1m 59s
635:	learn: 0.2714170	total: 3m 28s	remaining: 1m 59s
636:	learn: 0.2707617	total: 3m 28s	remaining: 1m 59s
637:	learn: 0.2702008	total: 3m 29s	remaining: 1m 58

772:	learn: 0.2179695	total: 4m 12s	remaining: 1m 14s
773:	learn: 0.2178398	total: 4m 12s	remaining: 1m 13s
774:	learn: 0.2175179	total: 4m 12s	remaining: 1m 13s
775:	learn: 0.2170366	total: 4m 13s	remaining: 1m 13s
776:	learn: 0.2165334	total: 4m 13s	remaining: 1m 12s
777:	learn: 0.2161573	total: 4m 13s	remaining: 1m 12s
778:	learn: 0.2158629	total: 4m 14s	remaining: 1m 12s
779:	learn: 0.2156882	total: 4m 14s	remaining: 1m 11s
780:	learn: 0.2152368	total: 4m 14s	remaining: 1m 11s
781:	learn: 0.2149933	total: 4m 15s	remaining: 1m 11s
782:	learn: 0.2147442	total: 4m 15s	remaining: 1m 10s
783:	learn: 0.2143678	total: 4m 15s	remaining: 1m 10s
784:	learn: 0.2140811	total: 4m 16s	remaining: 1m 10s
785:	learn: 0.2134130	total: 4m 16s	remaining: 1m 9s
786:	learn: 0.2131318	total: 4m 16s	remaining: 1m 9s
787:	learn: 0.2127892	total: 4m 17s	remaining: 1m 9s
788:	learn: 0.2125822	total: 4m 17s	remaining: 1m 8s
789:	learn: 0.2122874	total: 4m 17s	remaining: 1m 8s
790:	learn: 0.2119596	total: 4m 1

928:	learn: 0.1728038	total: 5m 2s	remaining: 23.1s
929:	learn: 0.1725952	total: 5m 3s	remaining: 22.8s
930:	learn: 0.1724470	total: 5m 3s	remaining: 22.5s
931:	learn: 0.1721828	total: 5m 3s	remaining: 22.2s
932:	learn: 0.1719513	total: 5m 4s	remaining: 21.8s
933:	learn: 0.1716768	total: 5m 4s	remaining: 21.5s
934:	learn: 0.1715879	total: 5m 4s	remaining: 21.2s
935:	learn: 0.1714067	total: 5m 5s	remaining: 20.9s
936:	learn: 0.1711856	total: 5m 5s	remaining: 20.5s
937:	learn: 0.1709959	total: 5m 5s	remaining: 20.2s
938:	learn: 0.1707955	total: 5m 5s	remaining: 19.9s
939:	learn: 0.1706031	total: 5m 6s	remaining: 19.6s
940:	learn: 0.1704183	total: 5m 6s	remaining: 19.2s
941:	learn: 0.1701837	total: 5m 6s	remaining: 18.9s
942:	learn: 0.1700562	total: 5m 7s	remaining: 18.6s
943:	learn: 0.1698339	total: 5m 7s	remaining: 18.2s
944:	learn: 0.1696864	total: 5m 7s	remaining: 17.9s
945:	learn: 0.1695018	total: 5m 8s	remaining: 17.6s
946:	learn: 0.1692755	total: 5m 8s	remaining: 17.3s
947:	learn: 

In [175]:
cat_model.score(psd_test_norm.reshape(n_ep_test, n_ch*n_f), labels_test.argmax(1))

0.3728813559322034

In [50]:
cat_model.predict(psd_test_norm.reshape(n_ep_test, n_ch*n_f))

array([[0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [1.],
       [0.],
       [2.],
       [2.],
       [1.],
       [2.],
       [2.],
       [0.],
       [1.],
       [1.],
       [0.],
       [2.],
       [2.],
       [1.],
       [2.],
       [1.],
       [2.],
       [1.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [1.],
       [2.],
       [1.],
       [2.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [1.],
       [2.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [2.],
       [1.],
       [0.],
       [1.],
       [0.]])